In [1]:
from discopy import *
from discopy.function import *

In [84]:
def scalar_mult(scalar):
     return Function('scalar_mult({})'.format(repr(scalar)), Dim(1), Dim(1), lambda x: scalar * x)
    
def scalar_mults(dom, weights):
    result = Id(0)
    for i in range(dom):
        result = result @ scalar_mult(weights[i])
    return result
   
def bias(scalar):
    return Function('bias({})'.format(repr(scalar)), Dim(0), Dim(1), lambda x: np.array([scalar]))

def merge(cod, copies):
    @discofunc(cod * copies, cod)
    def add(x):
        return np.array([np.sum([x[i + cod * j] for j in range(copies)]) for i in range(cod)])
    return add

def split(dom, copies):
    @discofunc(dom, dom * copies)
    def copy(x):
        return np.concatenate([x for i in range(copies)])
    return copy

@discofunc(1, 1)
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [85]:
def neuron(dom, cod, weights, beta=0): # weights is a 1d array of length dom, beta is a scalar bias
    return scalar_mults(dom, weights) @ bias(beta) >> merge(1, dom + 1) >> sigmoid >> split(1, cod)

In [86]:
neuron(4, 4, [0.5, 2.1, 0.3, 0.1])(np.array([0.1, 1, 0.2, 0.4]))

DeviceArray([0.9046505, 0.9046505, 0.9046505, 0.9046505], dtype=float32)

In [87]:
def layer(dom, cod, n_neurons, weights, biases): # weights is an array of size: n_neurons x dom
    neurons = Id(0)                              # biases is a 1d array of length n_neurons
    for i in range(n_neurons):
        neurons = neurons @ neuron(dom, cod, weights[i], biases[i])
    return split(dom, n_neurons) >> neurons >> merge(cod, n_neurons)